In [1]:
from cuda import cuda

def check_driver():
    # 1. Initialize the Driver
    err, = cuda.cuInit(0)
    if err != cuda.CUresult.CUDA_SUCCESS:
        print(f"❌ cuInit Failed: {err}")
        return

    # 2. Get Device Count
    err, count = cuda.cuDeviceGetCount()
    if err != cuda.CUresult.CUDA_SUCCESS:
        print(f"❌ GetCount Failed: {err}")
        return
    
    print(f"🟢 Driver Initialized. Found {count} Device(s).")

    # 3. Get Device Name
    for i in range(count):
        err, device = cuda.cuDeviceGet(i)
        err, name = cuda.cuDeviceGetName(128, device)
        # name is bytes, decode it
        print(f"   GPU {i}: {name.decode('utf-8').strip()}")
        
        # Check Compute Capability (Crucial for 5070/Blackwell)
        err, major = cuda.cuDeviceGetAttribute(
            cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR, device
        )
        err, minor = cuda.cuDeviceGetAttribute(
            cuda.CUdevice_attribute.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR, device
        )
        print(f"          Compute Capability: {major}.{minor}")

check_driver()

🟢 Driver Initialized. Found 1 Device(s).
   GPU 0: NVIDIA GeForce RTX 5070 Ti                                      
          Compute Capability: 12.0


<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.
